# MMC Implementation

## Correct Implementation

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import skmob
from tqdm import tqdm
import sys
sys.path.append('../')
from src.geo_utils import (get_clusters_from_tdf,
                            assign_tdf_points_to_clusters,
                            get_mmc_transitions,
                            get_stationary_vector)
import os

In [2]:
from src.geo_utils import (analyze_dates_with_events,
                           analyze_number_of_events, 
                           analyze_number_of_events_per_day, 
                           analyze_space_drift, 
                           analyze_speed_drift,
                           analyze_time_drift)

In [3]:
warnings.filterwarnings('ignore')

In [4]:
def get_distance(row):
    lat, lng, lat_last, lng_last = row['lat'], row['lng'], row['lat_last'], row['lng_last']
    return skmob.utils.utils.distance((lat, lng), (lat_last, lng_last))

### Evaluation of Implementation

In [ ]:
# Reading all the DataFrames and consolidating it into one
data_files = sorted([x for x in os.listdir('../data/') if 'geo_' in x])

data = pd.DataFrame()
for data_file in tqdm(data_files):
    data_i = pd.read_csv(f'../data/{data_file}')
    data = data.append(data_i)
data = data.reset_index(drop=True)

In [ ]:
# Homogenizing the dataset format

geo_columns = ['user', 'hour', 'lat', 'lng']

data['hour'] = data['date'] + ' ' + data['time']
data['hour'] = pd.to_datetime(data['hour'])
data['user'] = data['user_id'].map(int)

data = data[geo_columns]
data = data.drop_duplicates()
data = data.groupby(['user','hour'], as_index=False).nth(0)
data = data.reset_index(drop=True)


# Estimating distance and time differences

data[['lat_last','lng_last']] = data.groupby('user').shift(1)[['lat','lng']]
data['seconds_diff'] = data.groupby('user')['hour'].diff(1).dt.seconds

data['distance_to_last_km'] = data.apply(get_distance, axis=1) 
data['speed_mps'] = data['distance_to_last_km'] / data['seconds_diff'] * 1000

In [ ]:
data.info()

In [ ]:
del data['lat_last'], data['lng_last']

In [ ]:
data.head()

- El 99% de los registros están distanciados entre 0 y 5 segundos

#### Saving Data

In [ ]:
# Saving Data
data.to_parquet('../data/geolife_consolidated.parquet', index=False)

#### Loading Data (backup)

In [5]:
# Loading Data
data = pd.read_parquet('../data/geolife_consolidated.parquet')

In [6]:
data['date'] = data['hour'].dt.date

In [7]:
def analyze_time_drift(data):
    '''Analiza la distribución de la variación de tiempo (segundos) entre evento y evento'''
    
    # Distribución de número de eventos
    serie_seconds_events = data['seconds_diff'].value_counts()
    serie_seconds_events_log = np.log10(serie_seconds_events)

    print(serie_seconds_events)
    print()
    print(serie_seconds_events.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))
    fig.add_subplot(1,2,1)
    data['seconds_diff'].hist(bins=100)
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de Segundos')
    
    fig.add_subplot(1,2,2)

    serie_seconds_events_log.hist(bins=30)
    plt.title('Distribución de eventos según drift en segundos (log10)');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de Segundos (log10 scale)');
    

def analyze_space_drift(data):
    '''Analiza la distribución de la variación de distancia (Km) entre evento y evento'''

    serie_distance_events = data['distance_to_last_km'].value_counts()
    serie_distance_events_log = np.log10(serie_distance_events)
    
    print(serie_distance_events)
    print()
    print(serie_distance_events.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))
    fig.add_subplot(1,2,1)
    data['distance_to_last_km'].hist(bins=100)
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de KM');
    
    fig.add_subplot(1,2,2)

    serie_distance_events_log.hist(bins=30)
    plt.title('Distribución de eventos según drift de distancia (log10)');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Delta de KM (log10 scale)');
    
    
def analyze_speed_drift(data):
    '''Analiza la velocidad (metros por segundo) de cada evento'''

    serie_speed = data['speed_mps']
    serie_speed_log = np.log10(serie_speed+1)
    
    print(serie_speed.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))
    
    fig.add_subplot(1,2,1)
    serie_speed.hist(bins=100)
    plt.title('Distribución de eventos según velocidad');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Velocidad');
    
    fig.add_subplot(1,2,2)
    serie_speed_log.hist(bins=30)
    plt.title('Distribución de eventos según velocidad (log10)');
    plt.ylabel('Numero de Eventos')
    plt.xlabel('Velocidad (log10 scale)');
    
    
def analyze_number_of_events_per_day(data):
    '''Analiza la distribución de la variación de tiempo (segundos) entre evento y evento'''

    # Distribución de número de eventos
    df_events_per_day = data.groupby(['date','user'], as_index=False).size()
    
    serie_user_events_per_day = df_events_per_day['size']
    serie_user_events_per_day_log = np.log10(serie_user_events_per_day)

    print(serie_user_events_per_day)
    print()
    print(df_events_per_day['size'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))

    fig.add_subplot(1,2,1)
    serie_user_events_per_day.hist(bins=30)
    plt.title('Distribución de número de eventos por usuarios por día');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos en un día')    
    
    fig.add_subplot(1,2,2)
    serie_user_events_per_day_log.hist(bins=30)
    plt.title('Distribución de número de eventos por usuarios por día (log10)');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos en un día (log10 scale)')
    
def analyze_number_of_events(data):
    # Distribución de número de eventos
    serie_user_events = data['user'].value_counts()
    serie_user_events_log = np.log10(serie_user_events)

    print(serie_user_events)
    print()
    print(serie_user_events.quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]))

    fig = plt.figure(figsize=(15,4))

    fig.add_subplot(1,2,1)
    serie_user_events.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Eventos');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos')    
    
    fig.add_subplot(1,2,2)
    serie_user_events_log.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Eventos (log10)');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos (log10 scale)')
    

def analyze_dates_with_events(data):
    # Distribución de número de eventos
    serie_user_events = data[['user','date']].drop_duplicates()['user'].value_counts()
    serie_user_events_log = np.log10(serie_user_events)

    print(serie_user_events)

    fig = plt.figure(figsize=(15,4))
    fig.add_subplot(1,2,1)
    serie_user_events.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Días con Eventos');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de Días con eventos')
    
    fig.add_subplot(1,2,2)

    serie_user_events_log.hist(bins=30)
    plt.title('Distribución de usuarios según Nro de Días con Eventos (log10)');
    plt.ylabel('Numero de Usuarios')
    plt.xlabel('Número de eventos (log10 scale)')

In [ ]:
analyze_number_of_events(data)

In [ ]:
analyze_number_of_events_per_day(data)

In [ ]:
analyze_dates_with_events(data)

In [ ]:
analyze_space_drift(data)

In [ ]:
analyze_space_drift(data)

In [ ]:
analyze_speed_drift(data)

#### Latitud y Longitud

In [ ]:
data['lat'].hist(bins=50)

In [ ]:
data['lng'].hist(bins=50)

In [ ]:
data.shape

- Datashader

In [ ]:
dd = data.sample(20000)

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(data=dd, x='lng', y='lat')

In [ ]:
sns.kdeplot(data=dd, x='lat', y='lng', fill=True)

__Escenarios:__
- Base
- Filtro por país
- Filtro por tiempo en lugar

__Acuerdos__
- Quitar usuarios con menos de 1000 eventos
- Probar los 3 escenarios

In [8]:
def filter_min_events_per_user(_df, min_events, user_col = 'user'):
    print('Datos iniciales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    users_to_del = _df.groupby(user_col)[user_col].count()[_df.groupby(user_col)[user_col].count()<min_events].index
    _df = _df[~_df[user_col].isin(users_to_del)]
    
    print('Datos finales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    return _df 

In [9]:
data = filter_min_events_per_user(data, 1000)

Datos iniciales:
# Registros: 24,178,078 	| # Usuarios: 182
----------------------------------------------------------------------
Datos finales:
# Registros: 24,167,443 	| # Usuarios: 158
----------------------------------------------------------------------


In [10]:
def filter_min_days_per_user(_df, min_days, user_col = 'user', date_col  = 'date'):
    print('Datos iniciales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    users_to_del = _df.groupby(user_col)[date_col].nunique()[_df.groupby(user_col)[date_col].nunique()<min_days].index
    _df = _df[~_df[user_col].isin(users_to_del)]
    
    print('Datos finales:')
    print('# Registros: {:,}'.format(_df.shape[0]), '\t|', '# Usuarios: {}'.format(_df[user_col].nunique()))
    print('-'*70)
    return _df 

In [11]:
data = filter_min_days_per_user(data, 10)

Datos iniciales:
# Registros: 24,167,443 	| # Usuarios: 158
----------------------------------------------------------------------
Datos finales:
# Registros: 23,576,497 	| # Usuarios: 120
----------------------------------------------------------------------


In [ ]:
analyze_number_of_events_per_day(data)

In [ ]:
from skmob.preprocessing import (filtering, 
                                 detection, 
                                 compression, 
                                 clustering)

In [ ]:
# Creating a Trajectory DF to analyze the impact of filtering by velocity
trgeo = skmob.TrajDataFrame(
    data[data['user']==1], 
    datetime='hour',
    user_id='user'
)

In [ ]:
del trgeo
gc.collect()

In [ ]:
#Aproximadamente reducir a 10%

In [12]:
def get_mmc_clusters_stavectors(geo):
    '''Get MMC Clusters and Stationary Vectors
    
    Parameters
    ----------
        geo (pd.Dataframe): Dataframe of 1 user_id. Must contain only the columns: ['user', 'hour', 'lat', 'lng']
    
    Returns
    -------
        clusters (pd.Dataframe): Dataframe with the clusters and the stationary value
        m (folium.folium.Map): Map generated with the clusters on it.
    '''
    # TDF Definition
    trgeo = skmob.TrajDataFrame(
        geo, 
        datetime='hour',
        user_id='user'
    )

    # Cluster Generation
    clusters, m = get_clusters_from_tdf(
        trgeo,
        verbose=True,
        max_speed_kmh= 0.1,
        detect_stops=False,
        compress=False ,
        minutes_for_a_stop=2,
        spatial_radius_km=0.2,
        spatial_radius_compress_km=.2,
        cluster_radius_km=0.5,
        min_samples=2
    )

    # Cluster Assignation
    trgeo_cl, distances = assign_tdf_points_to_clusters(
        tdf=trgeo, 
        clusters=clusters
    )

    # Generation of Transit Dataframe
    transit_df = get_mmc_transitions(trgeo_cl)

    # Generation of Transit Matrix
    transit_matrix = pd.crosstab(transit_df['cluster'], 
                                 transit_df['cluster_next'],
                                 normalize='index').values
    display(transit_matrix)
    
################################## START FIX ANTHONY ################################

    #Lógica adicional para remover clusters sin salida o entrada
    ##Retiramos las filas que tienen solo ceros y una columna con 1
#    if ((transit_matrix.shape[0]>2 )& (transit_matrix.shape[1]>2)):
#        print('Revisando filas de la Matriz de transición...')
#        print('mostrando clusters iniciales')
#        display(clusters)
#        rows_to_drop = (transit_matrix==0).sum(axis=1)==(transit_matrix.shape[1]-1)#todos los valores en 0 excepto 1
#        rows_to_drop = [idx for idx,row in enumerate(rows_to_drop) if row]
#        if len(rows_to_drop)>0: 
#            transit_matrix = np.delete(transit_matrix,rows_to_drop,0)
#            clusters=clusters.drop(rows_to_drop, axis=0  )
#            print('clusters {} eliminados'.format(rows_to_drop))
#            print('Transit matrix actual:')
#            display(transit_matrix)
            
        ##Retiramos las columnas que tienen solo ceros una fila con 1
#        print('Revisando columnas de la Matriz de transición...')
#        print('mostrando clusters iniciales')
#        display(clusters)
#        cols_to_drop = (transit_matrix==0).sum(axis=0)==(transit_matrix.shape[0]-1)
#        cols_to_drop = [idx for idx,col in enumerate(cols_to_drop) if col]
#        if len(cols_to_drop)>0: 
#            transit_matrix = np.delete(transit_matrix,cols_to_drop,1)
#            clusters=clusters.drop(cols_to_drop, axis=0  )
#            print('clusters {} eliminados'.format(cols_to_drop))
#            print('Transit matrix actual:')
#            display(transit_matrix)
#    display(transit_matrix)

################################## END FIX ANTHONY ################################


    
    return clusters, m, transit_matrix, transit_df

In [13]:
# Extraction of clusters 
geo_clusters = {}
geo_clusters_img = {}
geo_clusters_transit = {}
geo_clusters_transit_df = {}
users = sorted(data['user'].unique())
for user in tqdm(users):

    geo = data[data['user']==user].reset_index(drop=True).copy()

    clusters, m, transit_matrix, transit_df = get_mmc_clusters_stavectors(geo)

    geo_clusters[user] = clusters
    geo_clusters_img[user] = m
    geo_clusters_transit[user] = transit_matrix
    geo_clusters_transit_df[user] = transit_df


  0%|                                                   | 0/120 [00:00<?, ?it/s]

INFO: Noise Filtering applied
INFO: Clusters generated
(173870, 5) (379, 5) (379, 5) (379, 5) (379, 6)
INFO: 12 clusters generated.



100%|███████████████████████████████████████████| 12/12 [00:31<00:00,  2.65s/it]


array([[0.        , 0.04347826, 0.26086957, 0.        , 0.04347826,
        0.30434783, 0.04347826, 0.13043478, 0.        , 0.        ,
        0.        , 0.17391304],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.5       , 0.125     , 0.        , 0.        , 0.        ,
        0.        , 0.125     , 0.        , 0.        , 0.25      ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.5       , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.27777778, 0.        , 0.05555556, 0.16666667,
        0.        ,

  1%|▎                                        | 1/120 [00:38<1:16:10, 38.41s/it]

...

 99%|████████████████████████████████████▋| 119/120 [21:42:08<01:43, 103.58s/it]

INFO: Noise Filtering applied
INFO: Clusters generated
(155312, 5) (74, 5) (74, 5) (74, 5) (74, 6)
INFO: 14 clusters generated.



100%|███████████████████████████████████████████| 14/14 [00:33<00:00,  2.42s/it]


array([[0.        , 0.54054054, 0.02702703, 0.        , 0.08108108,
        0.        , 0.        , 0.        , 0.18918919, 0.        ,
        0.        , 0.05405405, 0.05405405, 0.05405405],
       [0.5625    , 0.        , 0.        , 0.        , 0.39583333,
        0.        , 0.        , 0.        , 0.02083333, 0.        ,
        0.        , 0.02083333, 0.        , 0.        ],
       [0.05      , 0.        , 0.        , 0.05      , 0.15      ,
        0.05      , 0.        , 0.        , 0.        , 0.2       ,
        0.05      , 0.35      , 0.        , 0.1       ],
       [0.        , 0.        , 0.02702703, 0.        , 0.        ,
        0.        , 0.        , 0.62162162, 0.        , 0.35135135,
        0.        , 0.        , 0.        , 0.        ],
       [0.07692308, 0.51923077, 0.03846154, 0.        , 0.        ,
        0.        , 0.28846154, 0.01923077, 0.        , 0.        ,
        0.        , 0.03846154, 0.        , 0.01923077],
       [0.        , 0.        , 0. 

100%|█████████████████████████████████████| 120/120 [21:42:47<00:00, 651.39s/it]


In [14]:
import pickle

In [17]:
with open('../data/geo_clusters', 'wb') as f:
    pickle.dump(geo_clusters,f)

In [19]:
with open('../data/geo_clusters_transit', 'wb') as f:
    pickle.dump(geo_clusters_transit,f)

In [24]:
with open('../data/geo_clusters_transit_df', 'wb') as f:
    pickle.dump(geo_clusters_transit_df,f)

In [ ]:
# Extraction of clusters 
geo_clusters = {}
geo_clusters_img = {}
geo_clusters_transit = {}
geo_clusters_transit_df = {}
users = sorted(data['user'].unique())
for user in tqdm(users):

    geo = data[data['user']==user].reset_index(drop=True).copy()

    clusters, m, transit_matrix, transit_df = get_mmc_clusters_stavectors(geo)

    geo_clusters[user] = clusters
    geo_clusters_img[user] = m
    geo_clusters_transit[user] = transit_matrix
    geo_clusters_transit_df[user] = transit_df


In [ ]:
# Creating a Trajectory DF to analyze the impact of filtering by velocity
trgeo = skmob.TrajDataFrame(
    data[data['user']==10], 
    datetime='hour',
    user_id='user'
)

In [ ]:
# 1. Noise Filtering
print(trgeo.shape)
trgeo = filtering.filter(trgeo, max_speed_kmh=0.1)
print(trgeo.shape)

In [ ]:
for user in data[data['user']<10]['user'].unique():
    trgeo = skmob.TrajDataFrame(data[data['user']==user], datetime='hour',user_id='user')
    val1 = trgeo.shape[0]
    trgeo = filtering.filter(trgeo, max_speed_kmh=0.2)
    val2 = trgeo.shape[0]
    print('Puntos reducidos de {:,} a {:,}'.format(val1,val2))

In [ ]:
3305 / 108607

In [ ]:
print('a')

In [ ]:
aux_dict = {}
aux_dict['user'] = []
aux_dict['number_points'] = []
aux_dict['min_tmstmp'] = []
aux_dict['max_tmstmp'] = []
for usr in data['user'].unique():
    aux_dict['user'].append(usr)
    aux_dict['number_points'].append(data[data['user']==usr].shape[0])
    aux_dict['min_tmstmp'].append(data[data['user']==usr]['hour'].min())
    aux_dict['max_tmstmp'].append(data[data['user']==usr]['hour'].max())
aux_dict = pd.DataFrame(aux_dict)
aux_dict['duracion'] = (aux_dict['max_tmstmp'] - aux_dict['min_tmstmp']).dt.total_seconds()/ 60
aux_dict.sort_values(by='duracion', ascending=True, inplace=True)
display(aux_dict)

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
ax.set_xscale('log')
plt.scatter(aux_dict['duracion'].values, aux_dict['number_points'].values, color='r')
plt.plot(aux_dict['duracion'].values, aux_dict['number_points'].values , color='r', linestyle='--')
"""
plt.axvline(60,0)
plt.text(60,0, 'H')
plt.axvline(60*24,0)
plt.text(60*24,0, 'D')
"""
plt.axvline(60*24*7,0)
plt.text(60*24*7,0, 'S')
plt.axvline(60*24*7*30,0)
plt.text(60*24*7*30,0, 'M')
ax.set_xlabel('Tiempo en minutos - log')
ax.set_ylabel('Cantidad de puntos')
#plt.axvline(60*24*7*30*365,0)
#plt.text(60*24*7*30*365,0, 'Y')
#plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(12,6))
#ax.set_xscale('log')
plt.scatter(aux_dict['duracion'].values, aux_dict['number_points'].values)
plt.plot(aux_dict['duracion'].values, aux_dict['number_points'].values )

#plt.show()

In [ ]:
data['seconds_diff'].nlargest(30)

-------|-----|-----------|---------|

--xxXx--|--xX-x--|--x--xX--|

In [ ]:
data['distance_to_last_km'].nlargest(30)